In [ ]:
import gzip, json
with gzip.open('/data/NPMvulnerabilities/NPMPackages.json.gz', 'rb') as f:
   fc = f.read().decode("utf-8")
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
   while True:
       match = NOT_WHITESPACE.search(document, pos)
       if not match:
           return
       pos = match.start()

       try:
           obj, pos = decoder.raw_decode(document, pos)
       except JSONDecodeError:
           # do something sensible if there's some error
           print("DECODE_STACKED() FAILED")
           pass
       yield obj
    


In [ ]:
#pkgs = ['htmlnano', 'object.map', 'babylon-walk', 'wix-style-react', 'threads', 'minimist-options', 'lead', '@babel/polyfill', '@babel/helper-define-map', 'cloudscraper']
pkgs = ['ajv-keywords']
dependent = {}
for pkg in pkgs:
   dependent[pkg] = []
   for obj in decode_stacked(fc):
       ind = 0
       try: versions = list(obj['versions'].keys())
       except: continue
       try: release_dates = obj['time']
       except: continue
       info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
       for key in versions:
           try:
               info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
               if pkg in info[key]['dependencies']: ind = 1
           except: continue
       if ind :
           dependent[pkg].append({obj['name']})


In [ ]:
for k in dependent.keys():
   if k == 'ajv-keywords':
       print(k, dependent[k])
       print("len = ", len(dependent[k]))
       break